In [ ]:
# notebook in progress

training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)
trainer.train()

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Welches Wirtschaftszweig passt am besten zu dem unten beschriebenen Unternehmen?: Ticos Systems Deutschland GmbH entwickelt und bietet Lösungen für die Branche von Bädern und Wellness, Fitness, Kundenservice und Support. Das Unternehmen zeichnet sich durch seine innovative Software und Hardware aus, die Effizienz und Sicherheit von Bauprojekten verbessern soll. Der Zweck seiner Geschäftstätigkeit ist es, innovative Lösungen für die Baubranche zu entwickeln und anzubieten.?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False,
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True,
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150,
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])